<b> This script must be run in the same directory as the data folder containing the test and train folders. </b>

In [1]:
import os
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import pickle
from sklearn.metrics import f1_score

# In a python interpreter run the following:
# >>> nltk.download()

# code folding extension:
# pip install jupyter_contrib_nbextensions
# jupyter contrib nbextension install --user

In [2]:
# create list of words to be excluded from feature sets
stop_words = stopwords.words('english')
additional_stopwords = ['', 'isbn', 'edu']
stop_words.extend(additional_stopwords)

In [3]:
categories = {
    'alt': {
        'atheism': {}
    },
    'comp': {
        'graphics': {},
        'os': {
            'ms-windows': {
                'misc': {}
            }
        },
        'sys': {
            'ibm': {
                'pc': {
                    'hardware': {}
                }
            },
            'mac': {
                'hardware': {}
            }
        },
        'windows': {
            'x': {}
        }
    },
    'misc': {
        'forsale': {}
    },
    'rec': {
        'autos': {},
        'motorcycles': {},
        'sport': {
            'baseball': {},
            'hockey': {}
        }
    },
    'sci': {
        'crypt': {},
        'electronics': {},
        'med': {},
        'space': {}
    },
    'soc': {
        'religion': {
            'christian': {}
        }
    },
    'talk': {
        'politics': {
            'guns': {},
            'mideast': {},
            'misc': {}
        },
        'religion': {
            'misc': {}
        }
    }
}

# Utility functions to help navigate category tree
<br>

In [4]:
def get_root(category):
    root = categories.copy()
    if category.find('.') == -1:
        return {category : root[category]}
    branches = category.split('.')
    for branch in branches:
        try:
            root = root[branch]
        except TypeError:
            return root
    return {branch: root}

In [5]:
def traverse(root):
    if root is None or len(root.keys()) == 0:
        return
    if not isinstance(root, dict):
        return list(root)
    else:
        result = []
        for key in root.keys():
            parent = key
            children = traverse(root[key])
            if children is None:
                result.append(parent)
            else:
                for child in children:
                    result.append(parent + '.' + child)
        return result

In [6]:
def traverse_all():
    result = []
    for key in categories.keys():
        result.extend(traverse(get_root(key)))
    return result

# Functions to process raw emails and extract features
<br>

In [7]:
def extract_body(file_path, overwrite=True):
    if not overwrite:
        try:
            f = open(file_path + '.pickle', 'rb')
            parsed_file = pickle.load(f)
            f.close()
            return parsed_file
        except FileNotFoundError:
            pass
    
    f = open(file_path, 'r')
    count = 0
    while(True): 
        try:
            line = f.readline()
        except UnicodeDecodeError:
            continue
        if count > 150 or line.startswith('Lines:'):
            break
        count += 1

    document = dict()
    for line in f.readlines():
        alphanumeric_only = re.sub(r'\W+', ' ', line).strip(' ').lower()
        words = [word for word in alphanumeric_only.split(' ') if len(word) > 3 and not word.isnumeric() and word not in stop_words]
        for word in words:
            if word not in document.keys():
                document[word] = 1
            else:
                document[word] += 1
    f.close()

    if len(document) > 0:
        parsed_file = document
        f = open(file_path + '.pickle', 'wb')
        pickle.dump(parsed_file, f)
        f.close()
        return document
    else:
        return None

In [8]:
# given a string containing words separated by periods, return the substring
# containing the first i periods

def first_i_sections(string, num_periods):
    count = 0
    for i in range(len(string)):
        if count > num_periods:
            return string[:i].strip('.')
        if string[i] == '.':
            count += 1 
    return string

In [9]:
def get_documents(mode, category):
    base_path = './data/' + mode + '/'
    if category == 'all':
        folder_names = traverse_all()
    else:
        folder_names = traverse(get_root(category))
    documents = []
    
    for folder in folder_names:
        path = base_path + folder + '/'
        file_names = os.listdir(path)
        
        for file_name in file_names:
            file_path = path + file_name
            document = extract_body(file_path)
            if document is not None and len(document.keys()) > 0:
                documents.append(document)
    return documents

In [10]:
def get_labeled_documents(mode, category, full_category=False):
    periods = category.count('.')
    base_path = './data/' + mode + '/'
    if periods > 0:
        parent = first_i_sections(category, periods-1) + '.'
        base_path += parent
    else:
        parent = ''
    if category == 'all':
        folder_names = traverse_all()
    else:
        folder_names = traverse(get_root(category))
    documents = []
    
    for folder in folder_names:
        path = base_path + folder + '/'
        file_names = os.listdir(path)
        
        for file_name in file_names:
            file_path = path + file_name
            document = extract_body(file_path)
            if document is not None and len(document.keys()) > 0:
                if category == 'all':
                    if full_category:
                        tag = folder
                    else:
                        tag = first_i_sections(folder, 0)
                else:
                    tag = parent + first_i_sections(folder, periods+1)         
                documents.append((document, tag))
    return documents

In [11]:
# given a list of document where each document is a dictionary containing the
# number of occurences of each word, return the average frequency of each word among all documents.
# For example, if the word "cpu" occurs with 0.2 frequency in document 1 and 0.3 in document 2, its
# average frequency among both documents is 0.25.
def get_most_frequent_words(documents):
    dfs = []
    for document in documents:
        dfs.append(pd.Series(document))

    avg_frequency_distribution = dfs[0].copy()

    for df in dfs:
        avg_frequency_distribution.add(df, fill_value=0)

    avg_frequency_distribution /= len(dfs)
    
    return avg_frequency_distribution.sort_values(ascending=False)[:100].index.to_list()

In [12]:
def document_features(document):
    all_features_dict = {}
    for feature in all_features:
        all_features_dict[feature] = 0
    
    keys = all_features_dict.keys()
    for word in document:
        if word in keys:
            all_features_dict[word] += 1
    return all_features_dict

In [13]:
features_per_category = 300
try:
    f = open('all_features.pickle', 'rb')
    all_features = pickle.load(f)
except FileNotFoundError:
    all_features = []
    all_features.extend(get_most_frequent_words(get_documents('train', 'alt'))[:features_per_category])
    all_features.extend(get_most_frequent_words(get_documents('train', 'comp'))[:features_per_category])
    all_features.extend(get_most_frequent_words(get_documents('train', 'misc'))[:features_per_category])
    all_features.extend(get_most_frequent_words(get_documents('train', 'rec'))[:features_per_category])
    all_features.extend(get_most_frequent_words(get_documents('train', 'sci'))[:features_per_category])
    all_features.extend(get_most_frequent_words(get_documents('train', 'soc'))[:features_per_category])
    all_features.extend(get_most_frequent_words(get_documents('train', 'talk'))[:features_per_category])
    f = open('all_features.pickle', 'wb')
    pickle.dump(all_features, f)
f.close()

In [14]:
def get_data_set(mode, category, full_category=False):
    data_set = []
    if full_category:
        documents = get_labeled_documents(mode, category, full_category=True)
    else:
        documents = get_labeled_documents(mode, category)
    for document in documents:
        training_example = (document_features(document[0]), document[1])
        data_set.append(training_example)
    return data_set

In [15]:
def get_nodes():
    categories = traverse_all()
    result = set()
    for category in categories:
        sub_categories = category.split('.')
        for i in range(0, len(sub_categories)):
            if i == 0:
                tree_path = sub_categories[i]
            else:
                tree_path += '.' + sub_categories[i]
            result.add(tree_path)
    return list(result)

In [16]:
nodes_list = get_nodes()
nodes_list.sort()
nodes = {}
for node in nodes_list:
    nodes[node] = None

# Make root classifier
<br>

In [17]:
all_train_set = get_data_set('train', 'all')
all_test_set = get_data_set('test', 'all')

In [18]:
root_classifier = nltk.NaiveBayesClassifier.train(all_train_set)

In [19]:
acc = nltk.classify.accuracy(root_classifier, all_test_set)
print(f'Accuracy = {round(acc*100, 2)}%')

Accuracy = 53.57%


# Make local-level classifiers
<br>

In [20]:
for node in nodes.keys():
    train_set = get_data_set('train', node)
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    nodes[node] = classifier

In [21]:
for node in nodes.keys():
    test_set = get_data_set('test', node)
    classifier = nodes[node]
    acc = nltk.classify.accuracy(classifier, test_set)
    print(f'{node}'.ljust(30, ' ') + 'classifier accuracy = ' + f'{round(acc*100, 1)}%'.rjust(6, ' '))

alt                           classifier accuracy = 100.0%
alt.atheism                   classifier accuracy = 100.0%
comp                          classifier accuracy =  49.7%
comp.graphics                 classifier accuracy = 100.0%
comp.os                       classifier accuracy = 100.0%
comp.os.ms-windows            classifier accuracy = 100.0%
comp.os.ms-windows.misc       classifier accuracy = 100.0%
comp.sys                      classifier accuracy =  62.2%
comp.sys.ibm                  classifier accuracy = 100.0%
comp.sys.ibm.pc               classifier accuracy = 100.0%
comp.sys.ibm.pc.hardware      classifier accuracy = 100.0%
comp.sys.mac                  classifier accuracy = 100.0%
comp.sys.mac.hardware         classifier accuracy = 100.0%
comp.windows                  classifier accuracy = 100.0%
comp.windows.x                classifier accuracy = 100.0%
misc                          classifier accuracy = 100.0%
misc.forsale                  classifier accuracy = 100.

# Create classification pipeline

In [22]:
def categorize(document):
    current_prediction = root_classifier.classify(document)

    while True:
        #print(current_prediction)
        classifier = nodes[current_prediction]
        next_prediction = classifier.classify(document)
        if current_prediction == next_prediction:
            break
        else:
            current_prediction = next_prediction
    return current_prediction

In [23]:
all_test_set_fully_defined = get_data_set('test', 'all', full_category=True)

In [24]:
y_true = []
y_pred = []

for document in all_test_set_fully_defined:
    y_true.append(document[1])
    y_pred.append(categorize(document[0]))

In [25]:
f1 = f1_score(y_true, y_pred, average='macro')
print(f'f1 score = {round(f1, 3)}')

f1 score = 0.297
